In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df = pickle.load(open('data/drugbank_fingerprinted (1).pkl', 'rb'))
df = df[~pd.isna(df['target_gene_fingerprint'])]
df.head()

,drug_cas,drug_id,drug_name,target_actions,target_amino,target_gene,target_id,target_name,target_organism,pubchem_id,pubchem_name,target_gene_fingerprint,drug_fingerprint
31,128270-60-0,DB00006,Bivalirudin,[inhibitor],>lcl|BSEQ0016004|Prothrombin\nMAHVRGLQLPGCLALA...,>lcl|BSEQ0016005|Prothrombin (F2)\nATGGCGCACGT...,BE0000048,Prothrombin,Human,16129704,Bivalirudin,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,[0.10044218 0.18559723 -0.12865402 -0.0650997 ...
53,65807-02-5,DB00014,Goserelin,[agonist],>lcl|BSEQ0036957|Lutropin-choriogonadotropic h...,>lcl|BSEQ0010174|Lutropin-choriogonadotropic h...,BE0000134,Lutropin-choriogonadotropic hormone receptor,Human,5311128,goserelin,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,[0.1331214 0.2391842 -0.14361311 -0.14107014 0...
54,65807-02-5,DB00014,Goserelin,[agonist],>lcl|BSEQ0000405|Gonadotropin-releasing hormon...,>lcl|BSEQ0018926|Gonadotropin-releasing hormon...,BE0000203,Gonadotropin-releasing hormone receptor,Human,5311128,goserelin,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,[0.1331214 0.2391842 -0.14361311 -0.14107014 0...
131,16679-58-6,DB00035,Desmopressin,[agonist],>lcl|BSEQ0000583|Vasopressin V2 receptor\nMLMA...,>lcl|BSEQ0010065|Vasopressin V2 receptor (AVPR...,BE0000293,Vasopressin V2 receptor,Human,16051933,DESMOPRESSIN,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,[0.051722348 0.24374117 -0.16236018 -0.1723347...
132,16679-58-6,DB00035,Desmopressin,NaN,>lcl|BSEQ0009979|Vasopressin V1a receptor\nMRL...,>lcl|BSEQ0009980|Vasopressin V1a receptor (AVP...,BE0000165,Vasopressin V1a receptor,Human,16051933,DESMOPRESSIN,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,[0.051722348 0.24374117 -0.16236018 -0.1723347...


In [3]:
ndf = pd.read_csv('negative_samples/negative_samples_thrsh:12')[['node_0','node_1','shortest_path']]
# ndf10 = pd.read_csv('negative_samples/negative_samples_thrsh:10')[['node_1','node_2','path_len']]
# ndf8 = pd.read_csv('negative_samples/negative_samples_thrsh:8')[['node_1','node_2','path_len']]
#ndf = pd.read_csv('negative_samples/negative_samples_thrsh:6')[['node_1','node_2','path_len']]
#ndf = pd.read_csv('negative_samples/negative_samples_thrsh:4')[['node_1','node_2','path_len']]

In [4]:
def getData(x):
    return [float(i) for i in x[0].split(' ')]

In [5]:
df['target_gene_fingerprint'] = df['target_gene_fingerprint'].apply(getData)
df['drug_fingerprint'] = df['drug_fingerprint'].apply(getData)

In [6]:
data_1 = pd.concat([pd.DataFrame(df['target_gene_fingerprint'].values.tolist(), columns=['Target_Vec_'+str(i) for i in range(0, 100)]), 
               pd.DataFrame(df['drug_fingerprint'].values.tolist(), columns=['Drug_Vec_'+str(i) for i in range(0, 100)])], axis=1)
data_1['Class'] = 1

In [7]:
d = dict()
for item in np.unique(df['drug_id'].values):
    try:
        _ = d[item]
    except:
        d.update({item : df[df['drug_id'] == item]['drug_fingerprint'].values[0] })

for item in np.unique(df['target_id'].values):
    try:
        _ = d[item]
    except:
        d.update({item : df[df['target_id'] == item]['target_gene_fingerprint'].values[0] })

In [8]:
def getVec(n1, n2):
    try:
        if n1.startswith('DB'):
            drug = d[n1]
            target = d[n2]
        else:
            drug = d[n2]
            target = d[n1]
        return target, drug
    except:
        return np.nan, np.nan

In [9]:
ndf[['target_gene_fingerprint', 'drug_fingerprint']] = ndf.apply(lambda x: getVec(x.node_0, x.node_1), 
                                                         axis=1, result_type='expand')
ndf = ndf[~pd.isna(ndf['target_gene_fingerprint'])]
ndf = ndf.sample(data_1.shape[0])

In [10]:
data_2 = pd.concat([pd.DataFrame(ndf['target_gene_fingerprint'].values.tolist(), columns=['Target_Vec_'+str(i) for i in range(0, 100)]), 
               pd.DataFrame(ndf['drug_fingerprint'].values.tolist(), columns=['Drug_Vec_'+str(i) for i in range(0, 100)])], axis=1)

data_2['Class'] = 0

In [11]:
data = pd.concat([data_1, data_2])

In [13]:
pickle.dump(data,open("dataset_vBalanced.pkl", "wb"))